# Voice Conversion Flow Matching - Playground

This notebook allows quick experimentation to test the core hypothesis:

**Comparing:**
1. **Baseline CFM:** Standard Conditional Flow Matching starting from Gaussian Noise
2. **SG-Flow:** Rectified Flow starting from Orthogonally Projected Content Subspace

**Quick sanity checks:**
- Model architecture verification
- Loss computation
- Small-scale training
- Inference pipeline

In [ ]:
# Setup and imports
import os
import sys
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from pathlib import Path

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Import project modules
from config import Config
from models.flow_network import FlowNetwork
from models.flow_matching import BaselineCFM, SGFlow, create_flow_model
from models.decoder import WavLMToMelDecoder
from models.system import VoiceConversionSystem
from models.projection import OrthogonalProjection

config = Config()
print(f"Config loaded: WAVLM_DIM={config.WAVLM_DIM}, BATCH_SIZE={config.BATCH_SIZE}")

## 1. Verify Model Architecture

In [ ]:
# Create flow network
flow_net = FlowNetwork(
    d_input=config.WAVLM_DIM,  # 768
    d_model=512,
    d_spk=config.d_spk,  # 192
    num_layers=6,
    num_heads=8,
).to(device)

# Count parameters
num_params = sum(p.numel() for p in flow_net.parameters())
print(f"Flow Network parameters: {num_params:,} ({num_params/1e6:.2f}M)")

# Test forward pass
B, T, D = 2, 100, config.WAVLM_DIM
x_t = torch.randn(B, T, D).to(device)
t = torch.rand(B).to(device)
spk = torch.randn(B, config.d_spk).to(device)

with torch.no_grad():
    v = flow_net(x_t, t, spk)
print(f"Input shape: {x_t.shape}")
print(f"Output shape: {v.shape}")
print(f"Output matches input: {v.shape == x_t.shape}")

In [ ]:
# Create decoder (1D ResNet)
decoder = WavLMToMelDecoder(
    d_wavlm=config.WAVLM_DIM,
    d_spk=config.d_spk,
    d_hidden=512,
    n_mels=config.n_mels,
    num_layers=4,
).to(device)

num_params_dec = sum(p.numel() for p in decoder.parameters())
print(f"Decoder parameters: {num_params_dec:,} ({num_params_dec/1e6:.2f}M)")

# Test decoder forward pass
wavlm_feat = torch.randn(B, T, config.WAVLM_DIM).to(device)
spk_emb = torch.randn(B, config.d_spk).to(device)

with torch.no_grad():
    mel = decoder(wavlm_feat, spk_emb)
print(f"WavLM input: {wavlm_feat.shape}")
print(f"Mel output: {mel.shape}")
print(f"Expected ~{int(T * 1.25)} mel frames (1.25x upsample)")

## 2. Test Flow Matching Loss (Baseline CFM vs SG-Flow)

In [ ]:
# Create fake projection matrix for testing
# In real usage, this comes from SVD on WavLM features
D = config.WAVLM_DIM
k = config.svd_rank  # 64

# Create random orthogonal projection
torch.manual_seed(42)
random_matrix = torch.randn(D, k)
V_k, _ = torch.linalg.qr(random_matrix)
P_speaker = V_k @ V_k.T
P_content = torch.eye(D) - P_speaker
mean = torch.zeros(D)

# Save temporary projection matrix
os.makedirs("temp", exist_ok=True)
torch.save({
    "V_k": V_k,
    "P_speaker": P_speaker,
    "P_content": P_content,
    "mean": mean,
    "k": k,
}, "temp/projection_matrix.pt")
print(f"Created projection matrix: P_content shape = {P_content.shape}")
print(f"Speaker subspace rank: {k}")

In [ ]:
# Create both flow models
baseline_model = create_flow_model(
    method="baseline",
    d_input=config.WAVLM_DIM,
    d_model=256,  # Smaller for testing
    d_spk=config.d_spk,
    num_layers=4,
    num_heads=4,
).to(device)

sg_flow_model = create_flow_model(
    method="sg_flow",
    d_input=config.WAVLM_DIM,
    d_model=256,
    d_spk=config.d_spk,
    num_layers=4,
    num_heads=4,
    projection_path="temp/projection_matrix.pt",
).to(device)

print(f"Baseline CFM type: {type(baseline_model).__name__}")
print(f"SG-Flow type: {type(sg_flow_model).__name__}")

In [ ]:
# Test loss computation
B, T = 4, 50
x1 = torch.randn(B, T, config.WAVLM_DIM).to(device)  # Target features
spk = torch.randn(B, config.d_spk).to(device)  # Speaker embedding

# Compute losses
baseline_loss, baseline_info = baseline_model(x1, spk)
sg_flow_loss, sg_flow_info = sg_flow_model(x1, spk)

print("=" * 50)
print("LOSS COMPARISON (random init, single batch)")
print("=" * 50)
print(f"Baseline CFM Loss: {baseline_loss.item():.4f}")
print(f"SG-Flow Loss:      {sg_flow_loss.item():.4f}")
print(f"\nBaseline v_pred norm: {baseline_info['v_pred_norm']:.4f}")
print(f"SG-Flow v_pred norm:  {sg_flow_info['v_pred_norm']:.4f}")

## 3. Quick Training Loop (Synthetic Data)

In [ ]:
def create_synthetic_batch(batch_size, seq_len, device):
    """Create synthetic batch for testing."""
    return {
        "target_wavlm": torch.randn(batch_size, seq_len, config.WAVLM_DIM).to(device),
        "target_spk": torch.randn(batch_size, config.d_spk).to(device),
        "target_mel": torch.randn(batch_size, int(seq_len * 1.25), config.n_mels).to(device),
        "target_mask": torch.ones(batch_size, seq_len, dtype=torch.bool).to(device),
    }


def quick_train(model, optimizer, num_steps=100, batch_size=4, seq_len=50):
    """Quick training loop for sanity check."""
    model.train()
    losses = []
    
    pbar = tqdm(range(num_steps), desc="Training")
    for step in pbar:
        batch = create_synthetic_batch(batch_size, seq_len, device)
        
        optimizer.zero_grad()
        outputs = model.compute_loss(
            x1=batch["target_wavlm"],
            target_spk=batch["target_spk"],
            target_mel=batch["target_mel"],
            mask=batch["target_mask"],
        )
        
        loss = outputs["loss"]
        loss.backward()
        optimizer.step()
        
        losses.append(loss.item())
        pbar.set_postfix(loss=f"{loss.item():.4f}")
    
    return losses

In [ ]:
# Create smaller models for quick testing
def create_small_system(method):
    flow = create_flow_model(
        method=method,
        d_input=config.WAVLM_DIM,
        d_model=256,
        d_spk=config.d_spk,
        num_layers=2,
        num_heads=4,
        projection_path="temp/projection_matrix.pt" if method == "sg_flow" else None,
    )
    dec = WavLMToMelDecoder(
        d_wavlm=config.WAVLM_DIM,
        d_spk=config.d_spk,
        d_hidden=256,
        n_mels=config.n_mels,
        num_layers=2,
    )
    return VoiceConversionSystem(flow, dec).to(device)

# Create systems
baseline_system = create_small_system("baseline")
sg_flow_system = create_small_system("sg_flow")

print(f"Baseline system params: {sum(p.numel() for p in baseline_system.parameters()):,}")
print(f"SG-Flow system params: {sum(p.numel() for p in sg_flow_system.parameters()):,}")

In [ ]:
# Train both models
NUM_STEPS = 200

print("Training Baseline CFM...")
baseline_opt = optim.AdamW(baseline_system.parameters(), lr=1e-4)
baseline_losses = quick_train(baseline_system, baseline_opt, NUM_STEPS)

print("\nTraining SG-Flow...")
sg_flow_opt = optim.AdamW(sg_flow_system.parameters(), lr=1e-4)
sg_flow_losses = quick_train(sg_flow_system, sg_flow_opt, NUM_STEPS)

In [ ]:
# Plot training curves
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(baseline_losses, label="Baseline CFM", alpha=0.7)
plt.plot(sg_flow_losses, label="SG-Flow", alpha=0.7)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title("Training Loss (Synthetic Data)")
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
window = 20
baseline_smooth = np.convolve(baseline_losses, np.ones(window)/window, mode='valid')
sg_flow_smooth = np.convolve(sg_flow_losses, np.ones(window)/window, mode='valid')
plt.plot(baseline_smooth, label="Baseline CFM (smoothed)", linewidth=2)
plt.plot(sg_flow_smooth, label="SG-Flow (smoothed)", linewidth=2)
plt.xlabel("Step")
plt.ylabel("Loss")
plt.title(f"Smoothed Loss (window={window})")
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nFinal Baseline Loss: {baseline_losses[-1]:.4f}")
print(f"Final SG-Flow Loss: {sg_flow_losses[-1]:.4f}")

## 4. Test Inference (ODE Solver)

In [ ]:
def euler_ode_solve(flow_model, x0, spk_embed, num_steps=20):
    """Simple Euler ODE solver."""
    device = x0.device
    B = x0.shape[0]
    dt = 1.0 / num_steps
    x_t = x0.clone()
    
    trajectory = [x_t.clone()]
    
    for i in range(num_steps):
        t = torch.full((B,), i / num_steps, device=device)
        v = flow_model.velocity_net(x_t, t, spk_embed)
        x_t = x_t + v * dt
        trajectory.append(x_t.clone())
    
    return x_t, trajectory


# Test inference
sg_flow_system.eval()
with torch.no_grad():
    # Create source features
    source = torch.randn(1, 50, config.WAVLM_DIM).to(device)
    target_spk = torch.randn(1, config.d_spk).to(device)
    
    # Get starting point (content projection for SG-Flow)
    if hasattr(sg_flow_system.flow_model, 'projection'):
        x0 = sg_flow_system.flow_model.projection.project_content(source)
    else:
        x0 = source.clone()
    
    # Solve ODE
    x1, trajectory = euler_ode_solve(sg_flow_system.flow_model, x0, target_spk, num_steps=20)
    
    # Decode to mel
    mel = sg_flow_system.decoder(x1, target_spk)

print(f"Source shape: {source.shape}")
print(f"x0 (projected) shape: {x0.shape}")
print(f"x1 (after ODE) shape: {x1.shape}")
print(f"Mel output shape: {mel.shape}")
print(f"Trajectory length: {len(trajectory)} steps")

In [ ]:
# Visualize ODE trajectory (first 3 dimensions)
trajectory_np = torch.stack(trajectory).cpu().numpy()  # (steps, B, T, D)

plt.figure(figsize=(12, 4))

# Plot trajectory of first 3 dimensions at middle time step
mid_t = trajectory_np.shape[2] // 2
for dim in range(3):
    plt.subplot(1, 3, dim + 1)
    values = trajectory_np[:, 0, mid_t, dim]
    plt.plot(values, marker='o', markersize=3)
    plt.xlabel("ODE Step")
    plt.ylabel(f"Dim {dim}")
    plt.title(f"Dimension {dim} trajectory")
    plt.grid(True, alpha=0.3)

plt.suptitle(f"ODE Trajectory (t={mid_t}, first 3 dims)")
plt.tight_layout()
plt.show()

## 5. Training with Real Data (if preprocessed)

In [ ]:
# Check if preprocessed data exists
PREPROCESSED_DIR = "./preprocessed"
DATA_EXISTS = os.path.exists(os.path.join(PREPROCESSED_DIR, "metadata.json"))

if DATA_EXISTS:
    print(f"Found preprocessed data at {PREPROCESSED_DIR}")
    from data.dataset import create_dataloader
    
    train_loader, train_ds = create_dataloader(
        PREPROCESSED_DIR, "train", batch_size=4, num_workers=0
    )
    print(f"Training samples: {len(train_ds)}")
    
    # Get one batch
    batch = next(iter(train_loader))
    print(f"\nBatch keys: {batch.keys()}")
    print(f"target_wavlm shape: {batch['target_wavlm'].shape}")
    print(f"target_spk shape: {batch['target_spk'].shape}")
    print(f"target_mel shape: {batch['target_mel'].shape}")
else:
    print(f"No preprocessed data found at {PREPROCESSED_DIR}")
    print("\nTo preprocess LibriTTS dev-clean (~1.2GB, ~40 speakers), run:")
    print("  python data/preprocess.py --data_root ./data/LibriTTS/dev-clean --output_dir ./preprocessed")
    print("\nThen compute SVD projection:")
    print("  python utils/svd_projection.py --data_dir ./preprocessed --output ./preprocessed/projection_matrix.pt")

In [ ]:
# Quick training on real data (if available)
if DATA_EXISTS:
    print("Training on real data for 100 steps...")
    
    # Check for real projection matrix
    real_proj_path = os.path.join(PREPROCESSED_DIR, "projection_matrix.pt")
    if os.path.exists(real_proj_path):
        proj_path = real_proj_path
        print(f"Using real projection matrix: {proj_path}")
    else:
        proj_path = "temp/projection_matrix.pt"
        print(f"Using synthetic projection matrix: {proj_path}")
    
    # Create model with real/synthetic projection
    real_system = create_small_system("sg_flow")
    real_opt = optim.AdamW(real_system.parameters(), lr=1e-4)
    
    real_system.train()
    real_losses = []
    
    train_iter = iter(train_loader)
    for step in tqdm(range(100), desc="Real data training"):
        try:
            batch = next(train_iter)
        except StopIteration:
            train_iter = iter(train_loader)
            batch = next(train_iter)
        
        real_opt.zero_grad()
        outputs = real_system.compute_loss(
            x1=batch["target_wavlm"].to(device),
            target_spk=batch["target_spk"].to(device),
            target_mel=batch["target_mel"].to(device),
            mask=batch["target_mask"].to(device),
        )
        outputs["loss"].backward()
        real_opt.step()
        real_losses.append(outputs["loss"].item())
    
    plt.figure(figsize=(8, 4))
    plt.plot(real_losses)
    plt.xlabel("Step")
    plt.ylabel("Loss")
    plt.title("Training on Real Data")
    plt.grid(True, alpha=0.3)
    plt.show()
else:
    print("Skipping real data training (no preprocessed data)")

## 6. Cleanup

In [ ]:
# Clean up temporary files
import shutil
if os.path.exists("temp"):
    shutil.rmtree("temp")
    print("Cleaned up temp directory")

print("\n" + "="*60)
print("PLAYGROUND COMPLETE")
print("="*60)
print("\nNext steps (using LibriTTS dev-clean, ~1.2GB, ~40 speakers):")
print("")
print("1. Setup environment:")
print("   ./setup.sh")
print("")
print("2. Preprocess data:")
print("   python data/preprocess.py --data_root ./data/LibriTTS/dev-clean --output_dir ./preprocessed")
print("")
print("3. Compute SVD projection:")
print("   python utils/svd_projection.py --data_dir ./preprocessed --output ./preprocessed/projection_matrix.pt")
print("")
print("4. Train and compare:")
print("   python train.py --mode baseline --data_dir ./preprocessed  # Baseline CFM")
print("   python train.py --mode sg_flow --data_dir ./preprocessed   # SG-Flow (your method)")
print("")
print("5. Inference:")
print("   python inference.py --checkpoint checkpoints/sg_flow_best.pt --source_wav ... --ref_wav ...")